In [34]:
import pyspark
import findspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import *
sc=SparkContext.getOrCreate(SparkConf().setMaster("local[*]")) # '*' denotes maximum number of core processor used 
sqlcontext=SQLContext(sc)                            
                            

Creating a RDD reading a csv file with the help of SparkContext (sc).

In [35]:
input_file_path="file:///C:/Users/ckp43_000/Documents/movie_file/ratings.csv"

# creating RDD reading a csv file with the help of SparkContext.
lines_rdd=sc.textFile(input_file_path)

# Checking RDD has been created or not.
lines_rdd

file:///C:/Users/ckp43_000/Documents/movie_file/ratings.csv MapPartitionsRDD[20] at textFile at NativeMethodAccessorImpl.java:0

In [37]:
lines_rdd.collect()

['1,1,4.0,964982703',
 '1,3,4.0,964981247',
 '1,6,4.0,964982224',
 '1,47,5.0,964983815',
 '1,50,5.0,964982931',
 '1,70,3.0,964982400',
 '1,101,5.0,964980868',
 '1,110,4.0,964982176',
 '1,151,5.0,964984041',
 '1,157,5.0,964984100',
 '1,163,5.0,964983650',
 '1,216,5.0,964981208',
 '1,223,3.0,964980985',
 '1,231,5.0,964981179',
 '1,235,4.0,964980908',
 '1,260,5.0,964981680',
 '1,296,3.0,964982967',
 '1,316,3.0,964982310',
 '1,333,5.0,964981179',
 '1,349,4.0,964982563',
 '1,356,4.0,964980962',
 '1,362,5.0,964982588',
 '1,367,4.0,964981710',
 '1,423,3.0,964982363',
 '1,441,4.0,964980868',
 '1,457,5.0,964981909',
 '1,480,4.0,964982346',
 '1,500,3.0,964981208',
 '1,527,5.0,964984002',
 '1,543,4.0,964981179',
 '1,552,4.0,964982653',
 '1,553,5.0,964984153',
 '1,590,4.0,964982546',
 '1,592,4.0,964982271',
 '1,593,4.0,964983793',
 '1,596,5.0,964982838',
 '1,608,5.0,964982931',
 '1,648,3.0,964982563',
 '1,661,5.0,964982838',
 '1,673,3.0,964981775',
 '1,733,4.0,964982400',
 '1,736,3.0,964982653',
 

In above output we can see that output of rdd is in random format without any schema 

we cant perform most of the operation on this output format 

therefore we will give this rdd a schema where every records will be notified with a name 

for that we need to create a function which splits the values of rdd and assgign a name to each values

In [38]:
# Creating a function to provide a schema to RDD so that each element of the row will have a name.
def mapper(lines):
    fields=lines.split(",")
    return Row(user_id=int(fields[0]),movie_id=int(fields[1]),ratings=float(fields[2]),timestamp=int(fields[3]))

In [40]:
# Applying above schema on RDD with the help of 'map' function.
rating_rdd=lines_rdd.map(mapper)
rating_rdd.collect()

[Row(movie_id=1, ratings=4.0, timestamp=964982703, user_id=1),
 Row(movie_id=3, ratings=4.0, timestamp=964981247, user_id=1),
 Row(movie_id=6, ratings=4.0, timestamp=964982224, user_id=1),
 Row(movie_id=47, ratings=5.0, timestamp=964983815, user_id=1),
 Row(movie_id=50, ratings=5.0, timestamp=964982931, user_id=1),
 Row(movie_id=70, ratings=3.0, timestamp=964982400, user_id=1),
 Row(movie_id=101, ratings=5.0, timestamp=964980868, user_id=1),
 Row(movie_id=110, ratings=4.0, timestamp=964982176, user_id=1),
 Row(movie_id=151, ratings=5.0, timestamp=964984041, user_id=1),
 Row(movie_id=157, ratings=5.0, timestamp=964984100, user_id=1),
 Row(movie_id=163, ratings=5.0, timestamp=964983650, user_id=1),
 Row(movie_id=216, ratings=5.0, timestamp=964981208, user_id=1),
 Row(movie_id=223, ratings=3.0, timestamp=964980985, user_id=1),
 Row(movie_id=231, ratings=5.0, timestamp=964981179, user_id=1),
 Row(movie_id=235, ratings=4.0, timestamp=964980908, user_id=1),
 Row(movie_id=260, ratings=5.0, ti

##### creating a dataframe from RDD using 'SQLContext'.

In [42]:
rating_df=sqlcontext.createDataFrame(rating_rdd)

##### creating a dataframe from a RDD using SparkContext

      This will throw an error because there is no function in SparkContext to create a dataframe from a RDD

In [13]:
new_rating_df=sc.createDataFrame(rating_rdd)

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

in above example we can see that SparkContext has no attribute 'createDataFrame'

so to create data frame from RDD we need sql context

Check above:--->>>

In [44]:
rating_df.show(10)

+--------+-------+---------+-------+
|movie_id|ratings|timestamp|user_id|
+--------+-------+---------+-------+
|       1|    4.0|964982703|      1|
|       3|    4.0|964981247|      1|
|       6|    4.0|964982224|      1|
|      47|    5.0|964983815|      1|
|      50|    5.0|964982931|      1|
|      70|    3.0|964982400|      1|
|     101|    5.0|964980868|      1|
|     110|    4.0|964982176|      1|
|     151|    5.0|964984041|      1|
|     157|    5.0|964984100|      1|
+--------+-------+---------+-------+
only showing top 10 rows



In [45]:
rating_table=rating_df.createOrReplaceTempView("rating_table")

we can register the dataFrame 'rating_df' as a table then we can write sql query on the table and process

check above :--->>>

In [117]:
sqlcontext.sql("""
describe table rating_table""").show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
| movie_id|   bigint|   null|
|  ratings|   double|   null|
|timestamp|   bigint|   null|
|  user_id|   bigint|   null|
+---------+---------+-------+



In [47]:
# Using sparkSQL to write SQL query to get count of rating for each movie.
# SparkSQL script returns a dataframe. --  'top_rated_movies' will be a type of dataframe.

top_rated_movies=sqlcontext.sql("""
select movie_id,count(ratings) as cnt  from rating_table group by movie_id order by cnt desc limit 10 """)

In [48]:
# Checking whether dataframe has been created from SparkSQL script.
top_rated_movies

DataFrame[movie_id: bigint, cnt: bigint]

In [135]:
top_rated_movies.show()

+--------+---+
|movie_id|cnt|
+--------+---+
|     356|329|
|     318|317|
|     296|307|
|     593|279|
|    2571|278|
|     260|251|
|     480|238|
|     110|237|
|     589|224|
|     527|220|
+--------+---+



In [50]:
top_rated_movies.collect()

[Row(movie_id=356, cnt=329),
 Row(movie_id=318, cnt=317),
 Row(movie_id=296, cnt=307),
 Row(movie_id=593, cnt=279),
 Row(movie_id=2571, cnt=278),
 Row(movie_id=260, cnt=251),
 Row(movie_id=480, cnt=238),
 Row(movie_id=110, cnt=237),
 Row(movie_id=589, cnt=224),
 Row(movie_id=527, cnt=220)]

we can use '.collect' method on dataFrame to store the result of a rdd in list

results will be of type list since this is output of rdd

check above:-->>

In [56]:
data_f1=top_rated_movies.collect()

In [57]:
type(data_f1)

list

In [131]:
for movie in data_f1:
    print(movie)

Row(movie_id=356, cnt=329)
Row(movie_id=318, cnt=317)
Row(movie_id=296, cnt=307)
Row(movie_id=593, cnt=279)
Row(movie_id=2571, cnt=278)
Row(movie_id=260, cnt=251)
Row(movie_id=480, cnt=238)
Row(movie_id=110, cnt=237)
Row(movie_id=589, cnt=224)
Row(movie_id=527, cnt=220)
Row(movie_id=2959, cnt=218)
Row(movie_id=1, cnt=215)
Row(movie_id=1196, cnt=211)
Row(movie_id=50, cnt=204)
Row(movie_id=2858, cnt=204)
Row(movie_id=47, cnt=203)
Row(movie_id=780, cnt=202)
Row(movie_id=150, cnt=201)
Row(movie_id=1198, cnt=200)
Row(movie_id=4993, cnt=198)
Row(movie_id=1210, cnt=196)
Row(movie_id=858, cnt=192)
Row(movie_id=457, cnt=190)
Row(movie_id=592, cnt=189)
Row(movie_id=2028, cnt=188)
Row(movie_id=5952, cnt=188)
Row(movie_id=7153, cnt=185)
Row(movie_id=588, cnt=183)
Row(movie_id=608, cnt=181)
Row(movie_id=2762, cnt=179)
Row(movie_id=380, cnt=178)
Row(movie_id=32, cnt=177)
Row(movie_id=364, cnt=172)
Row(movie_id=377, cnt=171)
Row(movie_id=1270, cnt=171)
Row(movie_id=4306, cnt=170)
Row(movie_id=3578, c

Row(movie_id=4489, cnt=24)
Row(movie_id=86880, cnt=24)
Row(movie_id=7454, cnt=24)
Row(movie_id=1883, cnt=24)
Row(movie_id=66934, cnt=24)
Row(movie_id=6350, cnt=24)
Row(movie_id=531, cnt=24)
Row(movie_id=1409, cnt=23)
Row(movie_id=4161, cnt=23)
Row(movie_id=4235, cnt=23)
Row(movie_id=1256, cnt=23)
Row(movie_id=2336, cnt=23)
Row(movie_id=7099, cnt=23)
Row(movie_id=3827, cnt=23)
Row(movie_id=562, cnt=23)
Row(movie_id=933, cnt=23)
Row(movie_id=2496, cnt=23)
Row(movie_id=866, cnt=23)
Row(movie_id=1321, cnt=23)
Row(movie_id=1779, cnt=23)
Row(movie_id=2840, cnt=23)
Row(movie_id=2805, cnt=23)
Row(movie_id=105, cnt=23)
Row(movie_id=2052, cnt=23)
Row(movie_id=117529, cnt=23)
Row(movie_id=101, cnt=23)
Row(movie_id=1186, cnt=23)
Row(movie_id=81932, cnt=23)
Row(movie_id=45672, cnt=23)
Row(movie_id=1962, cnt=23)
Row(movie_id=46972, cnt=23)
Row(movie_id=37741, cnt=23)
Row(movie_id=1298, cnt=23)
Row(movie_id=3107, cnt=23)
Row(movie_id=2428, cnt=23)
Row(movie_id=44022, cnt=23)
Row(movie_id=799, cnt=23)

Row(movie_id=7802, cnt=8)
Row(movie_id=78574, cnt=8)
Row(movie_id=39381, cnt=8)
Row(movie_id=2613, cnt=8)
Row(movie_id=482, cnt=8)
Row(movie_id=2870, cnt=8)
Row(movie_id=4437, cnt=8)
Row(movie_id=7155, cnt=8)
Row(movie_id=2392, cnt=8)
Row(movie_id=544, cnt=8)
Row(movie_id=74, cnt=8)
Row(movie_id=8, cnt=8)
Row(movie_id=1707, cnt=8)
Row(movie_id=710, cnt=8)
Row(movie_id=5055, cnt=8)
Row(movie_id=1889, cnt=8)
Row(movie_id=55908, cnt=8)
Row(movie_id=3727, cnt=8)
Row(movie_id=1966, cnt=8)
Row(movie_id=2344, cnt=8)
Row(movie_id=57274, cnt=8)
Row(movie_id=8937, cnt=8)
Row(movie_id=2457, cnt=8)
Row(movie_id=177593, cnt=8)
Row(movie_id=433, cnt=8)
Row(movie_id=2752, cnt=8)
Row(movie_id=4037, cnt=8)
Row(movie_id=360, cnt=8)
Row(movie_id=7063, cnt=8)
Row(movie_id=79224, cnt=8)
Row(movie_id=8645, cnt=8)
Row(movie_id=71282, cnt=8)
Row(movie_id=102686, cnt=8)
Row(movie_id=50912, cnt=8)
Row(movie_id=2155, cnt=8)
Row(movie_id=51080, cnt=8)
Row(movie_id=2445, cnt=8)
Row(movie_id=92, cnt=8)
Row(movie_id

Row(movie_id=2736, cnt=5)
Row(movie_id=6538, cnt=5)
Row(movie_id=8607, cnt=5)
Row(movie_id=85401, cnt=5)
Row(movie_id=26242, cnt=5)
Row(movie_id=7700, cnt=5)
Row(movie_id=3310, cnt=5)
Row(movie_id=2888, cnt=5)
Row(movie_id=37739, cnt=5)
Row(movie_id=8372, cnt=5)
Row(movie_id=34334, cnt=5)
Row(movie_id=3946, cnt=5)
Row(movie_id=55814, cnt=5)
Row(movie_id=2073, cnt=5)
Row(movie_id=4104, cnt=5)
Row(movie_id=1043, cnt=5)
Row(movie_id=5876, cnt=5)
Row(movie_id=33158, cnt=5)
Row(movie_id=4030, cnt=5)
Row(movie_id=6301, cnt=5)
Row(movie_id=8827, cnt=5)
Row(movie_id=5451, cnt=5)
Row(movie_id=7649, cnt=5)
Row(movie_id=168418, cnt=5)
Row(movie_id=5665, cnt=5)
Row(movie_id=1993, cnt=5)
Row(movie_id=2937, cnt=5)
Row(movie_id=83803, cnt=5)
Row(movie_id=3286, cnt=5)
Row(movie_id=4474, cnt=5)
Row(movie_id=3401, cnt=5)
Row(movie_id=56921, cnt=5)
Row(movie_id=26258, cnt=5)
Row(movie_id=1006, cnt=5)
Row(movie_id=3723, cnt=5)
Row(movie_id=93272, cnt=5)
Row(movie_id=26084, cnt=5)
Row(movie_id=259, cnt=5)


Row(movie_id=98279, cnt=2)
Row(movie_id=2314, cnt=2)
Row(movie_id=5258, cnt=2)
Row(movie_id=8782, cnt=2)
Row(movie_id=78174, cnt=2)
Row(movie_id=5425, cnt=2)
Row(movie_id=3823, cnt=2)
Row(movie_id=6909, cnt=2)
Row(movie_id=7888, cnt=2)
Row(movie_id=7493, cnt=2)
Row(movie_id=88746, cnt=2)
Row(movie_id=1415, cnt=2)
Row(movie_id=56003, cnt=2)
Row(movie_id=25886, cnt=2)
Row(movie_id=3330, cnt=2)
Row(movie_id=168612, cnt=2)
Row(movie_id=6265, cnt=2)
Row(movie_id=107702, cnt=2)
Row(movie_id=1986, cnt=2)
Row(movie_id=4273, cnt=2)
Row(movie_id=4608, cnt=2)
Row(movie_id=5949, cnt=2)
Row(movie_id=142372, cnt=2)
Row(movie_id=2071, cnt=2)
Row(movie_id=3660, cnt=2)
Row(movie_id=93006, cnt=2)
Row(movie_id=90057, cnt=2)
Row(movie_id=3849, cnt=2)
Row(movie_id=5334, cnt=2)
Row(movie_id=53138, cnt=2)
Row(movie_id=139915, cnt=2)
Row(movie_id=5051, cnt=2)
Row(movie_id=26524, cnt=2)
Row(movie_id=170355, cnt=2)
Row(movie_id=6612, cnt=2)
Row(movie_id=53161, cnt=2)
Row(movie_id=62970, cnt=2)
Row(movie_id=5177

Row(movie_id=27769, cnt=1)
Row(movie_id=68650, cnt=1)
Row(movie_id=71484, cnt=1)
Row(movie_id=6644, cnt=1)
Row(movie_id=51666, cnt=1)
Row(movie_id=74370, cnt=1)
Row(movie_id=5746, cnt=1)
Row(movie_id=50806, cnt=1)
Row(movie_id=146986, cnt=1)
Row(movie_id=71970, cnt=1)
Row(movie_id=27816, cnt=1)
Row(movie_id=3345, cnt=1)
Row(movie_id=190219, cnt=1)
Row(movie_id=136816, cnt=1)
Row(movie_id=151311, cnt=1)
Row(movie_id=43333, cnt=1)
Row(movie_id=2362, cnt=1)
Row(movie_id=125914, cnt=1)
Row(movie_id=62718, cnt=1)
Row(movie_id=82152, cnt=1)
Row(movie_id=4715, cnt=1)
Row(movie_id=5054, cnt=1)
Row(movie_id=140852, cnt=1)
Row(movie_id=320, cnt=1)
Row(movie_id=2207, cnt=1)
Row(movie_id=47538, cnt=1)
Row(movie_id=80094, cnt=1)
Row(movie_id=130050, cnt=1)
Row(movie_id=2164, cnt=1)
Row(movie_id=72692, cnt=1)
Row(movie_id=136355, cnt=1)
Row(movie_id=121372, cnt=1)
Row(movie_id=6679, cnt=1)
Row(movie_id=31422, cnt=1)
Row(movie_id=131610, cnt=1)
Row(movie_id=882, cnt=1)
Row(movie_id=7987, cnt=1)
Row(m

Row(movie_id=61210, cnt=1)
Row(movie_id=63033, cnt=1)
Row(movie_id=4282, cnt=1)
Row(movie_id=1519, cnt=1)
Row(movie_id=118834, cnt=1)
Row(movie_id=79572, cnt=1)
Row(movie_id=136443, cnt=1)
Row(movie_id=158035, cnt=1)
Row(movie_id=61246, cnt=1)
Row(movie_id=5584, cnt=1)
Row(movie_id=100326, cnt=1)
Row(movie_id=133365, cnt=1)
Row(movie_id=144262, cnt=1)
Row(movie_id=1824, cnt=1)
Row(movie_id=8189, cnt=1)
Row(movie_id=5657, cnt=1)
Row(movie_id=53450, cnt=1)
Row(movie_id=35347, cnt=1)
Row(movie_id=579, cnt=1)
Row(movie_id=2897, cnt=1)
Row(movie_id=59915, cnt=1)
Row(movie_id=164280, cnt=1)
Row(movie_id=102666, cnt=1)
Row(movie_id=174045, cnt=1)
Row(movie_id=183959, cnt=1)
Row(movie_id=7979, cnt=1)
Row(movie_id=74226, cnt=1)
Row(movie_id=8998, cnt=1)
Row(movie_id=6021, cnt=1)
Row(movie_id=7745, cnt=1)
Row(movie_id=6036, cnt=1)
Row(movie_id=66090, cnt=1)
Row(movie_id=151777, cnt=1)
Row(movie_id=6788, cnt=1)
Row(movie_id=8458, cnt=1)
Row(movie_id=8617, cnt=1)
Row(movie_id=25769, cnt=1)
Row(mov

In [51]:
new_data=top_rated_movies.show(10)

+--------+---+
|movie_id|cnt|
+--------+---+
|     356|329|
|     318|317|
|     296|307|
|     593|279|
|    2571|278|
|     260|251|
|     480|238|
|     110|237|
|     589|224|
|     527|220|
+--------+---+



In [58]:
lines_rdd.count()

100836

In [59]:
type(lines_rdd)

pyspark.rdd.RDD

In [61]:
type(lines_rdd.collect())

list

In [62]:
# stop SparkContext.
sc.stop()